# Semantic Kernel Blog Writer Agent with Azure OpenAI
This notebook demonstrates how to use Semantic Kernel with Azure OpenAI and declarative YAML to create an agent that writes a blog post about a given topic.

In [ ]:
# Install required packages (uncomment if running interactively)
# !pip install semantic-kernel azure-identity pyyaml

## Define the YAML Agent Specification
The agent is defined using YAML, specifying its type, instructions, and input parameters.

In [1]:
import yaml

AGENT_YAML = '''
type: chat_completion_agent
name: BlogWriterAgent
description: An agent that generates blog posts about a given topic.
instructions: |
  Write a detailed blog post about {{$topic}}. The blog post should be approximately {{$length}} paragraphs long and cover key aspects of the topic.
model:
  id: gpt-4.1-mini
  options:
    temperature: 0.2
inputs:
  topic:
    description: The topic of the blog post.
    required: true
  length:
    description: The number of paragraphs in the blog post.
    required: true
    default: 5
template:
  format: semantic-kernel
'''

# Display the YAML for reference
print(yaml.safe_load(AGENT_YAML))

{'type': 'chat_completion_agent', 'name': 'BlogWriterAgent', 'description': 'An agent that generates blog posts about a given topic.', 'instructions': 'Write a detailed blog post about {{$topic}}. The blog post should be approximately {{$length}} paragraphs long and cover key aspects of the topic.\n', 'model': {'id': 'gpt-4.1-mini', 'options': {'temperature': 0.2}}, 'inputs': {'topic': {'description': 'The topic of the blog post.', 'required': True}, 'length': {'description': 'The number of paragraphs in the blog post.', 'required': True, 'default': 5}}, 'template': {'format': 'semantic-kernel'}}


## Set Up Azure OpenAI Credentials
You need to provide your Azure OpenAI endpoint and key. Replace the placeholders below with your actual values.

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT', '<your-endpoint>')
AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY', '<your-key>')
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME = os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME', '<your-deployment-id>')

## Create and Run the Blog Writer Agent
This cell creates the agent from the YAML spec and invokes it to write a blog post about a given topic.

In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.agents import AgentRegistry, ChatCompletionAgent, AzureResponsesAgent
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureOpenAISettings

# Create the Kernel and add the Azure OpenAI service (env vars will be picked up automatically)
# kernel = Kernel()
# kernel.add_service(AzureChatCompletion())

async def run_blog_writer(topic, length=5) -> str:
    agent_declaration = yaml.safe_load(AGENT_YAML)
    client = AzureResponsesAgent.create_client(
        # deployment_name="gpt-5-nano",
    )
    # Create the agent from YAML using the kernel and service
    # agent: AzureResponsesAgent = await AgentRegistry.create_from_yaml(
    #     yaml_str=AGENT_YAML, 
    #     client=client,
    #     # service=AzureChatCompletion()
    #     # kernel=kernel
    # )
    
    agent = AzureResponsesAgent(
        name=agent_declaration["name"],
        instructions=agent_declaration["instructions"],
        client=client,
        ai_model_id=agent_declaration["model"]["id"],
    )
    
    print(f"Using agent: {agent.name} with model: {agent.ai_model_id}")

    # Maintain conversation thread
    thread = None
    user_input = f"Write a blog post about {topic} that is {length} paragraphs long."
    response = await agent.get_response(
        messages=user_input,
        thread=thread,
        topic=topic,
        length=length
    )
    thread = response.thread
    return response.content

# Example usage
topic = "The role of AI Agents in BPMN"
length = 4

blog_article = await run_blog_writer(topic, length)

print("# Generated Blog Post:")
print(blog_article)

Using agent: BlogWriterAgent with model: gpt-4.1-mini
# Generated Blog Post:
**The Role of AI Agents in BPMN**

Business Process Model and Notation (BPMN) has long been a cornerstone for organizations aiming to visualize, analyze, and optimize their business processes. As artificial intelligence continues to advance, the integration of AI agents into BPMN frameworks is transforming how businesses automate workflows and make intelligent decisions. AI agents, acting as autonomous software entities, can interpret BPMN diagrams, execute tasks, and adapt processes dynamically based on real-time data, thereby elevating the traditional BPMN approach from static documentation to proactive process management.

One of the primary roles AI agents play in BPMN is enhancing process automation and efficiency. Unlike conventional automation tools that follow strict predefined rules, AI agents bring cognitive capabilities such as natural language understanding, machine learning, and predictive analyti

In [5]:
import yaml
from pydantic import BaseModel
from typing import List, Optional

class SEOAgentOutput(BaseModel):
    title: str
    meta_description: str
    slug: str
    h1: str
    h2s: List[str]
    revised_article: str
    improvements: List[str]
    seo_keywords: List[str]
    internal_links: List[str]
    external_links: List[str]
    readability_score: Optional[float]
    call_to_action: Optional[str]

SEO_AGENT_YAML = '''
type: chat_completion_agent
name: SEOOptimizerAgent
description: An agent that optimizes blog articles for SEO.
instructions: |
  Rewrite the following blog article to be SEO optimized. Use relevant keywords, improve readability, and ensure the content is engaging for search engines. Return a structured output containing:
    - title
    - meta description
    - slug
    - h1 and h2 headings
    - the revised article
    - a list of improvements
    - relevant SEO keywords
    - internal and external links
    - readability score
    - a call to action
  # Article
  {{$article}}
inputs:
  article:
    description: The original blog article to optimize.
    required: true
model:
  id: gpt-oss-120b
  options:
    temperature: 0.2
    response_format: SEOAgentOutput
template:
  format: semantic-kernel
'''

print(yaml.safe_load(SEO_AGENT_YAML))

{'type': 'chat_completion_agent', 'name': 'SEOOptimizerAgent', 'description': 'An agent that optimizes blog articles for SEO.', 'instructions': 'Rewrite the following blog article to be SEO optimized. Use relevant keywords, improve readability, and ensure the content is engaging for search engines. Return a structured output containing:\n  - title\n  - meta description\n  - slug\n  - h1 and h2 headings\n  - the revised article\n  - a list of improvements\n  - relevant SEO keywords\n  - internal and external links\n  - readability score\n  - a call to action\n# Article\n{{$article}}\n', 'inputs': {'article': {'description': 'The original blog article to optimize.', 'required': True}}, 'model': {'id': 'gpt-oss-120b', 'options': {'temperature': 0.2, 'response_format': 'SEOAgentOutput'}}, 'template': {'format': 'semantic-kernel'}}


In [ ]:
import json
from semantic_kernel import Kernel
from semantic_kernel.agents import AgentRegistry, ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureChatPromptExecutionSettings
from semantic_kernel.functions import KernelArguments

from pydantic import ValidationError

# Create the Kernel and add the Azure OpenAI service (env vars will be picked up automatically)
kernel = Kernel()
kernel.add_service(AzureChatCompletion())

# Create execution settings with structured output
execution_settings = AzureChatPromptExecutionSettings()
execution_settings.response_format = SEOAgentOutput

# Pass these settings as arguments when creating the agent
arguments = KernelArguments(settings=execution_settings)

async def run_seo_optimizer_structured(article):
    # Create the SEO optimizer agent from YAML
    agent: ChatCompletionAgent = await AgentRegistry.create_from_yaml(
        SEO_AGENT_YAML, kernel=kernel, arguments=arguments
    )

    # Maintain conversation thread
    thread = None
    response = await agent.get_response(
        # messages=article,
        thread=thread,
        article=article
    )
    thread = response.thread
    print(response.model_dump())
    return response.message.content

# Example usage
seo_blog_article_structured = await run_seo_optimizer_structured(blog_article)

# Parse structured output
try:
    result = SEOAgentOutput.model_validate(json.loads(seo_blog_article_structured))
    print("\n\n# SEO Optimization Result:")
    print(json.dumps(result.model_dump(), indent=2))
    print("\n\n# SEO Optimized Blog Article:")
    print(result.revised_article)
except ValidationError as e:
    print("Error parsing structured output:", e)
    print("Raw response:", seo_blog_article_structured)

{'message': {'ai_model_id': 'gpt-5-mini', 'metadata': {'logprobs': None, 'id': 'chatcmpl-C1zzDNh7KlFNNrWyU1hgnDrrRmuuj', 'created': 1754592639, 'system_fingerprint': None, 'usage': {'prompt_tokens': 858, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'completion_tokens': 1712, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 448, 'rejected_prediction_tokens': 0}}}, 'content_type': 'message', 'role': <AuthorRole.ASSISTANT: 'assistant'>, 'name': 'SEOOptimizerAgent', 'items': [{'ai_model_id': None, 'metadata': {}, 'content_type': 'text', 'text': '{"title":"How AI Agents Transform BPMN: Intelligent Workflows, Decisioning, and Process Mining","meta_description":"Discover how AI agents integrate with BPMN to create intelligent workflows—enhancing decisioning, automation, process mining, and governance for adaptive business processes.","slug":"ai-agents-bpmn-intelligent-workflows","h1":"How AI Agents Transform BPMN: Intel